In [1]:
# Install necessary packages
%pip install simpy matplotlib seaborn plotly

# Import libraries
import simpy
import random
import matplotlib.pyplot as plt
from collections import deque
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Constants (These can be parameterized further)
SIM_TIME = 10000  # Total simulation time in minutes
ARRIVAL_INTERVAL = 300  # Average time between vessel arrivals in minutes (5 hours)
CONTAINERS_PER_SHIP = 150  # Number of containers per vessel
CRANE_OPERATION_TIME = 3  # Time to move one container with a crane in minutes
TRUCK_CYCLE_TIME = 6  # Time for a truck to complete a round trip in minutes
TOTAL_BERTHS = 2  # Number of berths available
TOTAL_CRANES = 2  # Number of cranes available
TOTAL_TRUCKS = 3  # Number of trucks available

class PortTerminal:
    def __init__(self, env: simpy.Environment):
        self.env = env
        self.docks = simpy.Resource(env, capacity=TOTAL_BERTHS)
        self.lifting_cranes = simpy.Resource(env, capacity=TOTAL_CRANES)
        self.transport_trucks = simpy.Resource(env, capacity=TOTAL_TRUCKS)
        self.ship_queue = []  # Queue for waiting ships
        self.containers_transferred = 0  # Counter for moved containers
        self.cumulative_wait_time = 0  # Total wait time for vessels
        self.resource_mgr = ResourceManager(env, self)
        self.env_impact_tracker = EmissionsTracker(self)
        self.scheduler = ArrivalScheduler()

    def ship_arrival(self) -> None:
        """Process for vessel arrivals."""
        while True:
            time_until_next_arrival = self.scheduler.next_arrival_time()
            yield self.env.timeout(time_until_next_arrival)
            ship_id = f"Ship-{self.env.now}"
            print(f"{self.env.now}: {ship_id} has arrived")
            self.ship_queue.append(ship_id)
            self.env.process(self.process_ship(ship_id))

    def process_ship(self, ship_id: str) -> None:
        """Process for handling vessel berthing and container unloading."""
        arrival_timestamp = self.env.now
        with self.docks.request() as berth_request:
            yield berth_request
            waiting_duration = self.env.now - arrival_timestamp
            self.cumulative_wait_time += waiting_duration
            self.ship_queue.remove(ship_id)
            print(f"{self.env.now}: {ship_id} is berthed after waiting {waiting_duration} minutes")

            with self.lifting_cranes.request() as crane_request, self.resource_mgr.spare_cranes.request() as additional_crane_request:
                yield crane_request
                yield additional_crane_request
                for _ in range(CONTAINERS_PER_SHIP):
                    with self.transport_trucks.request() as truck_request, self.resource_mgr.spare_trucks.request() as additional_truck_request:
                        yield truck_request
                        yield additional_truck_request
                        yield self.env.timeout(CRANE_OPERATION_TIME)
                        self.containers_transferred += 1
                        print(f"{self.env.now}: {ship_id} - Container moved by crane")
                        self.env.process(self.truck_cycle())
                        self.env_impact_tracker.log_container_move()

            print(f"{self.env.now}: {ship_id} has departed")
            self.scheduler.update_log(time_until_next_arrival=waiting_duration)

    def truck_cycle(self) -> None:
        """Process for truck round trip."""
        yield self.env.timeout(TRUCK_CYCLE_TIME)
        self.env_impact_tracker.log_truck_trip()

class ResourceManager:
    def __init__(self, env: simpy.Environment, terminal: PortTerminal):
        self.env = env
        self.terminal = terminal
        self.spare_cranes = simpy.Resource(env, capacity=1)
        self.spare_trucks = simpy.Resource(env, capacity=2)
        self.env.process(self.manage_resources())

    def manage_resources(self) -> None:
        """Process to optimize resource allocation based on queue lengths."""
        while True:
            yield self.env.timeout(60)  # Check every hour
            self.allocate_cranes()
            self.allocate_trucks()

    def allocate_cranes(self) -> None:
        """Dynamically allocate cranes based on vessel queue length."""
        if len(self.terminal.ship_queue) > 1 and self.spare_cranes.level == 0:
            yield self.spare_cranes.put(1)  # Add a crane
            print(f"{self.env.now}: Added an extra crane. Total cranes: {self.terminal.lifting_cranes.capacity + self.spare_cranes.level}")
        elif len(self.terminal.ship_queue) == 0 and self.spare_cranes.level > 0:
            yield self.spare_cranes.get(1)  # Remove a crane
            print(f"{self.env.now}: Removed a crane. Total cranes: {self.terminal.lifting_cranes.capacity + self.spare_cranes.level}")

    def allocate_trucks(self) -> None:
        """Dynamically allocate trucks based on their utilization."""
        if self.terminal.transport_trucks.count == self.terminal.transport_trucks.capacity + self.spare_trucks.level and self.spare_trucks.level < 2:
            yield self.spare_trucks.put(1)  # Add a truck
            print(f"{self.env.now}: Added an extra truck. Total trucks: {self.terminal.transport_trucks.capacity + self.spare_trucks.level}")
        elif self.terminal.transport_trucks.count < (self.terminal.transport_trucks.capacity + self.spare_trucks.level) // 2 and self.spare_trucks.level > 0:
            yield self.spare_trucks.get(1)  # Remove a truck
            print(f"{self.env.now}: Removed a truck. Total trucks: {self.terminal.transport_trucks.capacity + self.spare_trucks.level}")

class EmissionsTracker:
    def __init__(self, terminal: PortTerminal):
        self.terminal = terminal
        self.total_emissions = 0
        self.crane_operations = 0
        self.truck_runs = 0

    def log_container_move(self) -> None:
        """Record emissions for container moves."""
        self.crane_operations += 1
        self.total_emissions += 0.5  # Assuming 0.5 units of emissions per crane move

    def log_truck_trip(self) -> None:
        """Record emissions for truck trips."""
        self.truck_runs += 1
        self.total_emissions += 1  # Assuming 1 unit of emissions per truck trip

    def get_total_emissions(self) -> float:
        """Get total emissions."""
        return self.total_emissions

class ArrivalScheduler:
    def __init__(self):
        self.history = deque(maxlen=100)
        self.base_interval = ARRIVAL_INTERVAL

    def next_arrival_time(self) -> float:
        """Predict the next vessel arrival time based on historical data."""
        if len(self.history) < 10:
            return random.expovariate(1/self.base_interval)
        return random.expovariate(1/np.mean(self.history))

    def update_log(self, time_until_next_arrival: float) -> None:
        """Update historical data with the latest inter-arrival time."""
        self.history.append(time_until_next_arrival)

class Dashboard:
    def __init__(self, terminal: PortTerminal):
        self.terminal = terminal
        self.ship_queue_log = []
        self.containers_transferred_log = []
        self.emissions_log = []
        self.resource_usage_log = []
        self.crane_operations_log = []
        self.truck_runs_log = []
        self.wait_times_log = []

    def update_logs(self) -> None:
        """Update the dashboard data."""
        self.ship_queue_log.append(len(self.terminal.ship_queue))
        self.containers_transferred_log.append(self.terminal.containers_transferred)
        self.emissions_log.append(self.terminal.env_impact_tracker.get_total_emissions())
        berth_util = self.terminal.docks.count / self.terminal.docks.capacity
        crane_util = self.terminal.lifting_cranes.count / (self.terminal.lifting_cranes.capacity + self.terminal.resource_mgr.spare_cranes.capacity)
        truck_util = self.terminal.transport_trucks.count / (self.terminal.transport_trucks.capacity + self.terminal.resource_mgr.spare_trucks.capacity)
        self.resource_usage_log.append((berth_util, crane_util, truck_util))
        self.crane_operations_log.append(self.terminal.env_impact_tracker.crane_operations)
        self.truck_runs_log.append(self.terminal.env_impact_tracker.truck_runs)
        self.wait_times_log.append(self.terminal.cumulative_wait_time / (len(self.terminal.ship_queue) + 1))

    def plot_results(self) -> None:
        """Plot the simulation results."""
        fig = make_subplots(rows=3, cols=2, subplot_titles=(
            'Ship Queue Over Time', 'Containers Transferred Over Time', 'Total Emissions Over Time',
            'Resource Usage Over Time', 'Distribution of Ship Wait Times', 'Crane and Truck Activity Over Time'
        ))

        fig.add_trace(go.Scatter(y=self.ship_queue_log, mode='lines', name='Ship Queue Length'), row=1, col=1)
        fig.add_trace(go.Scatter(y=self.containers_transferred_log, mode='lines', name='Containers Transferred'), row=1, col=2)
        fig.add_trace(go.Scatter(y=self.emissions_log, mode='lines', name='Total Emissions'), row=2, col=1)

        berth_usage, crane_usage, truck_usage = zip(*self.resource_usage_log)
        fig.add_trace(go.Scatter(y=berth_usage, mode='lines', name='Berth Utilization'), row=2, col=2)
        fig.add_trace(go.Scatter(y=crane_usage, mode='lines', name='Crane Utilization'), row=2, col=2)
        fig.add_trace(go.Scatter(y=truck_usage, mode='lines', name='Truck Utilization'), row=2, col=2)

        fig.add_trace(go.Histogram(x=self.wait_times_log, name='Ship Wait Times'), row=3, col=1)

        fig.add_trace(go.Scatter(y=self.crane_operations_log, mode='lines', name='Crane Operations'), row=3, col=2)
        fig.add_trace(go.Scatter(y=self.truck_runs_log, mode='lines', name='Truck Runs'), row=3, col=2)

        fig.update_layout(height=800, width=1200, title_text='Port Terminal Simulation Results')
        fig.show()

# Simulation setup and execution
env = simpy.Environment()
port_terminal = PortTerminal(env)
dashboard = Dashboard(port_terminal)

env.process(port_terminal.ship_arrival())

# Run simulation and update dashboard
for _ in range(SIM_TIME):
    env.run(until=env.now + 1)
    dashboard.update_logs()

# Plot results
dashboard.plot_results()


93.50554549471134: Ship-93.50554549471134 has arrived
93.50554549471134: Ship-93.50554549471134 is berthed after waiting 0.0 minutes
96.50554549471134: Ship-93.50554549471134 - Container moved by crane
99.50554549471134: Ship-93.50554549471134 - Container moved by crane
102.50554549471134: Ship-93.50554549471134 - Container moved by crane
105.50554549471134: Ship-93.50554549471134 - Container moved by crane
107.41223237024552: Ship-107.41223237024552 has arrived
107.41223237024552: Ship-107.41223237024552 is berthed after waiting 0.0 minutes
108.50554549471134: Ship-93.50554549471134 - Container moved by crane
111.50554549471134: Ship-93.50554549471134 - Container moved by crane
114.50554549471134: Ship-93.50554549471134 - Container moved by crane
117.50554549471134: Ship-93.50554549471134 - Container moved by crane
120.50554549471134: Ship-93.50554549471134 - Container moved by crane
123.50554549471134: Ship-93.50554549471134 - Container moved by crane
126.50554549471134: Ship-93.5055